# WeRateDogs Analysis

## Steps involved in Wrangling
- Gather
- Assess
- Clean


## Gather
In this step we obtain data for each of the three different pieces of data in Jupyter Notebook.

#### Enhanced Twitter Archive
    This file has been provided to us as a csv file.
    
1. We used read_csv function as below to load the file and create df<br />
    twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')<br />
    twitter_archive_enhanced.sample()*Verify*

#### Additional Data via the Twitter API
    As the basic Twitter metrics: retweet count and favorite count were missing from Enhanced Twitter Archive Data, we'll query and gather this data through Twitter API
1. Setup twitter connection<br />
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)<br />
auth.set_access_token(access_token, access_secret)<br />
api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
2. Create list of tweet_ids from Enhanced Twitter Archive data.<br />
tweet_id_list=twitter_archive_enhanced.tweet_id.to_list()
3. Gather data from Twitter API.<br />
tweets, drop_lst = [], []<br />
start = time.time()<br />
for id in tweet_id_list:<br />
    try:<br />
        tweet = api.get_status(id,tweet_mode='extended')<br />
        tweets.append(tweet._json)
        
    except tweepy.TweepError as e:
        error = e.args[0][0]['message']
        drop_lst.append({'tweet_id':id,
                        'error':error})
    end = time.time()<br />
    *To monitor total time taken for gathering data from Twitter API<br />*
    print(end - start)<br />
4. Write data to a text file<br />
    with open('twee_json.txt', 'w') as fh:<br />
    fh.write('\n'.join(json.dumps(tweet) for tweet in tweets))
5. Read data from text file and create dataframe<br />
    with open("twee_json.txt") as fh:<br />
    tweets = [json.loads(line) for line in fh if line]<br />
    twitter_api_data = pd.DataFrame(tweets)<br />
    twitter_api_data.sample()
6. Create dataframe of all tweet_ids for which twitter data wasn't found.<br />
    t_api_unfound_data = pd.DataFrame(drop_lst,columns=['tweet_id','error'])<br />
    t_api_unfound_data.sample()

#### Image Predictions File
    Data from running all the images through a neural network. 
    This data is hosted on Udacity's servers and needs to be downloaded programmatically using the Requests library. 
1. Downloading data and reading the .tsv file and creating dataframe<br />
    url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'<br />
    download = requests.get(url)<br />
    with open(url.split('/')[-1], mode = 'wb') as outfile:<br />
     outfile.write(download.content)

    image_predictions = pd.read_csv('image-predictions.tsv', sep = '\t', encoding = 'utf-8')<br />
    image_predictions.head()

## Assess
This is the second step in data wrangling process. In this step we assess data for two types of issues:- <br />
- Quality *issues with content.*
 - Examples:- incorrect datatypes, data outliers, etc. 
- Tidiness *issues with structure that prevent easy analysis.*
 - Examples:- Multple tables for one row, multiple columns. 
 
#### Enhanced Twitter Archive
- twitter_archive_enhanced.shape[0] *To get columns,rows of the dataframe*
- twitter_archive_enhanced.info() *To get quick overview of the dataframe*
- twitter_archive_enhanced.sample() *To view a random row of the dataframe*
- type(twitter_archive_enhanced['timestamp'][0]) *To get the data type of the column* __Incorrect datatype__
- twitter_archive_enhanced.name.value_counts() *To view count associated with different Name values in dataframe* __Incorrect enteries such as 'a', 'an', 'the' & 'None'__
- twitter_archive_enhanced.rating_denominator.value_counts() *To view count associated with different Denominator values in dataframe* __Per the description of the project Denominator Ratings should be 10 but here we have outliers__
- twitter_archive_enhanced[twitter_archive_enhanced['retweeted_status_id'].notnull()].head() *To view rows in dataframe with not null values in Retweeted_Status_ID column* 
#### Image Predictions File
- image_predictions.shape[0] *To get columns,rows of the dataframe*
- image_predictions.sample() *To view a random row of the dataframe*
- image_predictions.info() *To get quick overview of the dataframe*
- image_predictions.describe() *To view basic statistical details such as percentile, mean, std etc. of a dataframe.*
- image_predictions.describe(include='all') *To include unique, top, freq of string columns along with basic statistics of numeric columns.*
- image_predictions.nunique() *To view unique rows in dataframe.*
- image_predictions.head() *To view top 5 rows in dataframe.* __From this we can identify '_' in names of the breed precitions with two words is a quality issue and mixture of cases in the breed names, sometimes lowercase, etc is a quality issue too.__
- sum(image_predictions.duplicated()) *To get count of duplicate rows in dataframe.*
- sum(image_predictions.tweet_id.duplicated()) *To get count of duplicate rows in dataframe based on __tweet_id__ column.*
- sum(image_predictions.jpg_url.duplicated()) *To get count of duplicate rows in dataframe based on __jpg_url__ column.* __We found 66 duplicate rows.__
- duplicateRows_image_predictions = image_predictions[image_predictions.duplicated(['jpg_url'],keep=False)] *Create df with duplicate rows based on __jpg_url__ column.*
- duplicateRows_image_predictions.sort_values(by='jpg_url').head(10) *View top 10 rows of duplicate rows to do viual assessment to identify any Quality issue.* __We found duplicate tweets of same data which could skew our anlysis. This is a quality issue.__
#### Twitter API Data
- twitter_api_data.shape[0] *To get columns,rows of the dataframe*
- twitter_api_data.info() *To get quick overview of the dataframe*
- twitter_api_data.sample() *To view a random row of the dataframe* __Downloaded data has toomany columns which we don't need for analysis i.e. a quality issue__

#### Quality
##### `Enhanced Twitter Archive` table
1. Erroneous datatype timestamp
2. Remove Retweets based on Retweeted Status Id
3. Drop columns retweeted_status_id,retweeted_status_user_id & retweeted_status_timestamp
4. Name column has incorrect data.
5. ratings_denominator have outliers. 
6. Reorganising dog_stages data into different buckets

##### `Image Predictions` table
7. '_' in the breed predicted columns. 
8. Unsymmetric casetypes in breed predicted columns.
9. Remove duplicate tweets based on same jpg_url

##### `Twitter API Data` table
10. Drop unnecessary columns

#### Tidiness
1. One variable in four columns in `Enchanced Twitter Archive` table (dog_stage)
2. retweet count and favorite count should be part of `Enchanced Twitter Archive`
3. There should be one combined dataset for each tweet_id

## Clean
This is the third step in data wrangling process. In this step we fixed the __Quality__ & __Tidiness__ issues identified in __Assess__ step.
 
### Quality
##### Create copy of dataframes so initial dataframes remains unaffected.

tae_clean = twitter_archive_enhanced.copy()<br />
image_pred_clean = image_predictions.copy()<br />
tad_clean = twitter_api_data.copy()<br />
#### Enhanced Twitter Archive

###### 1. Erroneous datatype timestamp
*remove microseconds from timestamp and convert column timestamp to timestamp datatype*<br />
tae_clean['timestamp'] =pd.to_datetime(tae_clean.timestamp, format="%Y-%m-%d %H:%M:%S +%f")*Converting to timestamp datatype.<br /> 
tae_clean['timestamp'] = tae_clean.timestamp[:-5] *Removing microseconds*<br />
tae_clean.sample() *Verify*<br />

###### 2. Remove Retweets based on Retweeted Status Id
tae_clean = tae_clean[tae_clean.retweeted_status_id.isnull()] *Recreating df from df excluding rows where rewteeted_status_id is Null as we want to exclude rewteeted ids* <br />
tae_clean.info() *Verify*

###### 3. Drop columns retweeted_status_id,retweeted_status_user_id & retweeted_status_timestamp
tae_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],axis=1,inplace=True) *Dropping columns retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp*<br />
tae_clean.info() *Verify*

###### 4. Name column has incorrect data.
*create df with lowercase name values and uppercase name case values*
tae_clean_lowercase_name = tae_clean[tae_clean['name'].str[0].str.islower()]<br />
tae_clean_uppercase_name = tae_clean[tae_clean['name'].str[0].str.isupper()]<br />

*View value count of dfs based on Name column.*
    tae_clean_lowercase_name.name.value_counts()<br />
    tae_clean_uppercase_name.name.value_counts()<br />
*__As we can visually assess, lowercase Name values are all incorrect.__*<br />
    
*Remove Lowercase values from Name column*<br />
    tae_clean = tae_clean[~tae_clean['name'].str[0].str.islower()]<br />
    tae_clean.name.value_counts() *Verify*

###### 5. ratings_denominator have outliers. 
*Drop rows which have rating_denominator not equal to 10.*<br />
tae_clean = tae_clean[tae_clean['rating_denominator']==10] *Removing rows which have ratings_denominator __Not Equal__ to __10__.*<br />
tae_clean.rating_denominator.value_counts() *Verify*

#### Image Predictions File

###### 7. '_' in the breed predicted columns. 
*Replace " _ " from __p1, p2, p3__ coulumn values.*<br />
image_pred_clean['p1'] = image_pred_clean['p1'].str.replace('_ ', ' ') <br />
image_pred_clean['p2'] = image_pred_clean['p2'].str.replace('_ ', ' ')<br />
image_pred_clean['p3'] = image_pred_clean['p3'].str.replace('_ ', ' ')<br />

image_pred_clean.head() *Verify*

###### 8. Unsymmetric casetypes in breed predicted columns.
*Fix random case for values of __p1, p2, p3__ coulumns to Title Case.*<br />
image_pred_clean['p1'] = image_pred_clean['p1'].str.title()<br />
image_pred_clean['p2'] = image_pred_clean['p2'].str.title() <br />
image_pred_clean['p3'] = image_pred_clean['p3'].str.title() <br />

image_pred_clean.head() *Verify*

###### 9. Remove duplicate tweets based on same jpg_url.
image_pred_clean[image_pred_clean.duplicated(['jpg_url'],keep=False)].sort_values(by='jpg_url').head(6) <br />
__Based on above output we can visually assess duplicates__

*Drop duplicate rows*<br />
image_pred_clean.drop_duplicates(['jpg_url'],inplace=True)<br />
image_pred_clean[image_pred_clean.duplicated(['jpg_url'],keep=False)].sort_values(by='jpg_url').head(6) *Verify*

#### Twitter API Data

###### 10. Drop unnecessary columns
*Create column with tweet_id, favorite_count & retweet_count. 
We need Tweet_id to join the dataframe with Enhanced Twitter Archive and we need favorite count and retweet count as they're unavailable in Enhanced Twitter Archive dataframe.*

tad_clean = tad_clean.filter(['id','favorite_count','retweet_count'],axis=1)<br />
tad_clean.info() *Verify*

### Tidiness

###### 1. One variable in four columns in `Enchanced Twitter Archive` table (dog_stage)
*Melt the doggo, floofer, pupper, puppo columns to a dog_stage column.*

*Replace None with '' in puppo column*<br />
tae_clean['puppo'].replace('None','',inplace=True)<br />
tae_clean.puppo.value_counts()

*Replace None with '' in doggo column*<br />
tae_clean['doggo'].replace('None','',inplace=True)<br />
tae_clean.doggo.value_counts()

*Replace None with '' in pupper column*<br />
tae_clean['pupper'].replace('None','',inplace=True)<br />
tae_clean.doggo.value_counts()

*Replace None with '' in floofer column*<br />
tae_clean['floofer'].replace('None','',inplace=True)<br />
tae_clean.doggo.value_counts()

*Melt*<br />
tae_clean['dog_stages'] = tae_clean['doggo']+tae_clean['floofer']+tae_clean['pupper']+tae_clean['puppo'] <br />
tae_clean.dog_stages.value_counts() *Verify*

#### Quality 
###### 6. Reorganising dog_stages data into different buckets
Based on **The Dogtionary** puppo, pupper & doggo are stages and floofer can be accredited with any dog with fur. 
Hence, it makes sense to make following replacements:- 
- doggopupper with doggo who's just behaving like a  pupper
- doggopuppo with puppo as it's advanced stage of doggo
- doggofloofer with doggo 
- floofer with none as it is not a stage, just any dog really. 

tae_clean['dog_stages'].replace('doggopupper','doggo',inplace=True)<br />
tae_clean['dog_stages'].replace('doggofloofer','doggo',inplace=True)<br />
tae_clean['dog_stages'].replace('doggopuppo','puppo',inplace=True)<br />
tae_clean['dog_stages'].replace('floofer','',inplace=True)<br />
tae_clean.dog_stages.value_counts() *Verify*

*Replace '' with NaN in dog_stages column for cleaner data storage.*<br />
tae_clean['dog_stages'].replace('',np.nan,inplace=True)<br />
tae_clean.dog_stages.value_counts() *Verify*

*Drop columns doggo, floofer, pupper, puppo*<br />
tae_clean.drop(['doggo','floofer','pupper','puppo'],axis=1,inplace=True)<br />
tae_clean.info() *Verify*

### Tidiness
###### 2. retweet count and favorite count should be part of `Enchanced Twitter Archive`
*Merge retweet count and favorite count from __`Twitter API Data`__ with __`Enchanced Twitter Archive`__*

*Rename column id to tweet_id in Twitter API Data copy*<br />
tad_clean.rename(columns = {'id':'tweet_id'},inplace=True)<br />
tad_clean.info()

*Merging Twitter API data with Enhanced Twitter Archive data.*<br />
tae_clean = pd.merge(tae_clean,tad_clean, on='tweet_id',how='left')<br />
tae_clean.info() *Verify*

###### 3. There should be one combined dataset for each tweet_id
*Merging Enhanced Twitter Archive data with Image Predictions data. 
We'll be doing a right join between Enhanced Twitter Archive data & Image Predictions data to anlyze only those tweets for which we have image predictions data.*

tweet_data = pd.merge(tae_clean,image_pred_clean, on='tweet_id',how='right') *Creating holistic dataframe*

tweet_data.info() *Verify*

## Store
In this step we store the cleaned data.

tweet_data.to_csv("twitter_archive_master.csv",index=False)